In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import wandb

In [ ]:
# Parameters
sampling_method = 'desugar'
#sampling_method = 'graph'
use_wandb = False
%env WANDB_NOTEBOOK_NAME='analysis.ipynb'

In [ ]:
# Calculations
if use_wandb: 
    wandb.init(project="test-project", entity="cs532-2022")

In [ ]:
# Definitions
def plot_dist(data, nr, nc, names=None, panel_size=5., verbose=True, use_wandb=False, wandb_name=None):
    samples = data.shape[0]
    n = data.shape[1]
    fig, _ = plt.subplots(figsize=(nc*panel_size, nr*panel_size))
    for i in range(n):
        plt.subplot(nr, nc, 1+i)
        mean = data[:, i].mean()
        std = data[:, i].std()
        if verbose:
            print('Mean:', mean)
            print('Std:', std)
        plt.hist(data[:, i], bins='auto', color='C%d'%i)
        plt.axvline(mean, color='black', ls='--', label='Mean: %1.2f'%mean)
        plt.axvline(mean-std, color='black', ls=':', label='Std: %1.2f'%std)
        plt.axvline(mean+std, color='black', ls=':')
        if names is not None: plt.xlabel(names[i])
        plt.yticks([])
        plt.legend()
    plt.suptitle('Samples: %d'%samples)
    plt.tight_layout()
    plt.show()
    if use_wandb: wandb.log({wandb_name: wandb.Image(fig)})

def plot_traces(data, nr, nc, names=None, panel_size=5., verbose=False, use_wandb=False, wandb_name=None):
    samples = data.shape[0]
    n = data.shape[1]
    fig, _ = plt.subplots(figsize=(nc*panel_size, nr*panel_size))
    for i in range(n):
        plt.subplot(nr, nc, 1+i)
        mean = data[:, i].mean()
        std = data[:, i].std()
        if verbose:
            print('Mean:', mean)
            print('Std:', std)
        plt.plot(data[:, i], color='C%d'%i, alpha=0.3)
        plt.scatter(list(range(samples)), data[:, i], color='C%d'%i, marker='.')#, alpha=0.3)
        plt.axhline(mean, color='black', ls='--', label='Mean: %1.2f'%mean)
        plt.axhline(mean-std, color='black', ls=':', label='Std: %1.2f'%std)
        plt.axhline(mean+std, color='black', ls=':')
        if names is not None: plt.ylabel(names[i])
        plt.legend()
        plt.xlabel('samples')
        plt.xlim(left=0.)
    plt.suptitle('Samples: %d'%samples)
    plt.tight_layout()
    plt.show()
    if use_wandb: wandb.log({wandb_name: wandb.Image(fig)})

def plot_heatmap(data, use_wandb=False, wandb_name=None):
    xs = np.linspace(0, data.shape[1]-1, num=data.shape[1])
    x = []; y = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            x.append(xs[j])
            y.append(data[i, j])
    fig, _ = plt.subplots(figsize=(19, 3))
    plt.title('Number of samples: %d'%data.shape[0])
    plt.hist2d(x, y, bins=(17, 3), range=[[-0.5, 16.5], [-0.5, 2.5]], vmin=0., vmax=data.shape[0], density=False, cmap='binary')
    plt.xticks(list(range(17)))
    plt.xlabel('State')
    plt.yticks([0, 1, 2])
    plt.ylabel('Value')
    plt.colorbar(label='Number of samples')
    plt.show()
    if use_wandb: wandb.log({wandb_name: wandb.Image(fig)})

In [ ]:
# Program 1
variables = [r'$\mu$']
print('Program: 1')
data = np.loadtxt('data/homework_2/1_%s.dat'%(sampling_method))
data = np.atleast_2d(data).T
print('Data shape:', data.shape)
plot_dist(data, nr=1, nc=data.shape[1], names=variables, use_wandb=use_wandb, wandb_name='Program: 1')
plot_traces(data, nr=1, nc=data.shape[1], names=variables, use_wandb=use_wandb, wandb_name='Samples: 1')

In [ ]:
# Program 2
variables = ['slope', 'bias']
print('Program: 2')
data = np.loadtxt('data/homework_2/2_%s.dat'%(sampling_method))
print('Data shape:', data.shape)
plot_dist(data, nr=1, nc=data.shape[1], names=variables, use_wandb=use_wandb, wandb_name='Program: 2')
plot_traces(data, nr=1, nc=data.shape[1], names=variables, use_wandb=use_wandb, wandb_name='Samples: 2')

In [ ]:
# Program 3
data = np.loadtxt('data/homework_2/3_'+sampling_method+'.dat')
plot_heatmap(data, use_wandb=use_wandb, wandb_name='Program: 3')

In [ ]:
# Program 4
data = np.loadtxt('data/homework_2/4_'+sampling_method+'.dat')
print('Data shape:', data.shape)

# W0
variables = ['W_{0,%d)'%(i) for i in range(10)]
plot_dist(data[:, 0:10], nr=1, nc=10, names=variables, panel_size=3, verbose=False, use_wandb=use_wandb, wandb_name='Program 4: W0')

# b0
variables = ['b_{0,%d)'%(i) for i in range(10)]
plot_dist(data[:, 10:20], nr=1, nc=10, names=variables, panel_size=3, verbose=False, use_wandb=use_wandb, wandb_name='Program 4: b0')

# W1
variables = []
for i in range(10):
    for j in range(10):
        variables.append('W_{1,%d%d)'%(i, j))
plot_dist(data[:, 20:120], nr=10, nc=10, names=variables, panel_size=3, verbose=False, use_wandb=use_wandb, wandb_name='Program 4: W1')

# b1
variables = ['b_{1,%d)'%(i) for i in range(10)]
plot_dist(data[:, 120:130], nr=1, nc=10, names=variables, panel_size=3, verbose=False, use_wandb=use_wandb, wandb_name='Program 4: b1')

if use_wandb: wandb.finish()